# Tracing Basics

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

### Tracing with @traceable

In [13]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are a physics assistant. You will be required for answering theoretical questions based on certain formulae. Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


@traceable handles the RunTree lifecycle for you!

In [15]:
question = "What is the value of the gravitational constant?"
ai_answer = langsmith_rag(question)
print(ai_answer)

The value of the gravitational constant, denoted as \( G \), is approximately \( 6.674 \times 10^{-11} \, \text{m}^3 \, \text{kg}^{-1} \, \text{s}^{-2} \). This constant is used in the calculation of gravitational force between two masses. It plays a crucial role in Newton's law of universal gravitation.


### Adding Metadata

In [ ]:
from langsmith import traceable

@traceable(
    metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [16]:
question = "If I shoot a ball at 37 degrees to the ground going 5 meters per second, how far from me will it land?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To find the horizontal distance the ball will land, you can use the projectile motion equations. First, calculate the time of flight using the vertical component of the initial velocity and then use that time to find the horizontal distance. The horizontal distance \(d\) can be calculated as \(d = v_x \cdot t\), where \(v_x = v \cdot \cos(\theta)\) and \(t\) is the time of flight.


Adding metadata at runtime:

In [18]:
question = "What is torque?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "physics"}})
print(ai_answer)

Torque is a measure of the rotational force applied to an object, typically calculated as the product of the force and the distance from the pivot point (lever arm). The formula for torque (τ) is τ = r × F × sin(θ), where r is the distance from the pivot, F is the applied force, and θ is the angle between the force vector and the lever arm. Torque is a vector quantity, meaning it has both magnitude and direction.
